<a href="https://colab.research.google.com/github/Alimumtaz95/Q2_assignments/blob/main/exploring_gemini_2_0_video_and_audio_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import google.generativeai as genai

from google.colab import userdata
os.environ["Gemini_API_Key"] = userdata.get('Gemini_API_Key')

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
  system_instruction="Answer the user question based on the provided video",
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("/content/drive/MyDrive/Videos/intro_video.mp4", mime_type="video/mp4"),
]

# Some files have a processing delay. Wait for them to be ready.
wait_for_files_active(files)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    },
    {
      "role": "user",
      "parts": [
        "Generate a transcript",
      ],
    },
    {
      "role": "model",
      "parts": [
        "The person in the video introduces himself as Mumtaz Ali. He states that he is 29 years old and studying BBAIT at the Virtual University of Pakistan. He also mentions that he is a student of AJNTKAI from PIIC. Finally, he adds that he loves to code.",
      ],
    },
  ]
)

response = chat_session.send_message("Explain What the guy is doing also provide the trascript in urdu in every 2 seconds")

print(response.text)

Uploaded file 'intro_video.mp4' as: https://generativelanguage.googleapis.com/v1beta/files/s9iyjjljs5eq
Waiting for file processing...
...all files ready

Okay, here's a breakdown of what the man in the video is doing, along with a transcript in both English and Urdu, updated every 2 seconds:

**What the Guy is Doing:**

The man is recording a short introductory video. He is speaking directly to the camera, looking at the lens as if he is talking to the viewer. He’s sharing his name, age, what he is currently studying, and a general interest that he has. It seems like he may be making this video for social media or for some kind of personal introduction.

**Transcript with Timestamps and Urdu:**

**[00:00-00:02]**
*   **English:** "Assalam o Alaikum, this is Mumtaz Ali."
*   **Urdu:**  "السلام علیکم، یہ ممتاز علی ہے۔"

**[00:02-00:04]**
*   **English:** “Uh, I'm 29.”
*   **Urdu:** "میں 29 سال کا ہوں۔"

**[00:04-00:06]**
*   **English:** “Currently I’m doing BBAIT from Virtual Universit